In [1]:
%%HTML
<p id = 'heading1_p'> Provisioning - PIC32CKSG</p>
<hr id = 'hline'> </hr> 
<p id = 'heading1_p' style='font-size: 20px; font-weight: bold;'>Introduction</p>
<div id='div_element'>
<p> This usecase will provision the keys to the selected slot</p>
</div>

<button class="collapsible" style='font-size: 20px; font-weight: bold;'>Cryptographic Asset list & Use Case implementation</button>
<div class="content">
<div id='content_element'>
<hr id = 'h_content'> </hr> 
<p style="float: left; font-size: 9pt; text-align: center; 20em; width: 55%; margin-right: 1%;"><img src="assets/images/provision_steps.png" style="height: 60em; width: 100%"><b>Use Case implementation</b></p>
<p style="clear: both;">
</div>
</div>

<div id='div_element'>
<p>Select your Evaluation kit and then click on each step to execute the Use Case. Click on Usecase Help button in transaction diagram for additional details.</p>
</div>

In [ ]:
import os, shutil
from tpds.usecase_diagram import ProtoBoard, UsecaseDiagram
from tpds.tp_utils.tp_settings import TPSettings
from tpds.proto_boards import get_board_path
from IPython.display import display

curr_path = os.path.abspath('')
os.sys.path.extend([os.path.join(os.getcwd(), 'assets', 'python')])
import provision

tp_settings = TPSettings()
usecase_working_dir = os.path.join(tp_settings.get_base_folder(), 'pic32ck_sg01_provisioning')
os.makedirs(usecase_working_dir, exist_ok=True)
boards = ProtoBoard(max_width=400)
combined_hex_path=os.path.join(curr_path,'..','backend','boards','PIC32CKSG01', 'pic32cksg01_kit_image.hex')
shutil.copy(combined_hex_path, usecase_working_dir)

boards.add_board(
            'PIC32CKSG01 CUltra', 
            os.path.join(curr_path,'..','backend','boards','PIC32CKSG01','PIC32CKSG01.png'), 
            os.path.join(curr_path,'..','backend','boards','PIC32CKSG01', 'pic32cksg01_kit_image.hex'),           
            os.path.join(curr_path, 'firmware','NonSecure', 'firmware', 'pic32ck_sg_kitprotocol_hsm_NonSecure.X')
            )
boards.render_boards(0)
            
boards.render_boards(0)

#copy account setup yaml to users folder
#shutil.copy(os.path.join(curr_path, 'PIC32CXSG61_app.hex'), usecase_working_dir)

sec_boot = provision.ProvisioningUsecase(boards)
usecase_td = UsecaseDiagram(os.path.join(curr_path,'assets','images','provisioning.png'), working_dir=usecase_working_dir)

#1
kit_protocol_hex = usecase_td.add_script(205,105,35,30, sec_boot.connect_to_SE)

#2
generate_resources = usecase_td.add_script(205,170,35,30, sec_boot.generate_resources_asymmetric_key)#prereq_scripts = [kit_protocol_hex]

#3
write_key = usecase_td.add_script(205,240,35,30, sec_boot.write_slot) #prereq_scripts = [generate_resources]

#4
read_key= usecase_td.add_script(205,295,35,30,sec_boot.read_slot)#,prereq_scripts = [write_key ])

#5
delete_key =  usecase_td.add_script(205,358,35,30, sec_boot.delete_slot)#, prereq_scripts = [read_key])

#sub_process_program
program_combined_hex = usecase_td.add_script(650,105,35,30, sec_boot.flash_combined_firmware)

#sub_process_readslotinfo
read_slotinfo= usecase_td.add_script(475,295,35,30, sec_boot.read_slotinfo)


usecase_td.add_firmware(boards.get_firmware_project, tp_settings.get_mplab_paths().get('ide_path'))
usecase_td.add_image(726,262,141,27, os.path.join(curr_path, 'assets', 'images', 'c_snippet_template.PNG'), 'Code Snippet')
usecase_td.add_usecase_help(os.path.join(curr_path, 'assets', 'pic32ck_sg-provisioning.md'))

display(boards.canvas)
usecase_td.display_canvas()

MultiCanvas(height=200, layout=Layout(margin='50px'), width=200)